# LangChain 核心模块学习：Model I/O

`Model I/O` 是 LangChain 为开发者提供的一套面向 LLM 的标准化模型接口，包括模型输入（Prompts）、模型输出（Output Parsers）和模型本身（Models）。

- Prompts：模板化、动态选择和管理模型输入
- Models：以通用接口调用语言模型
- Output Parser：从模型输出中提取信息，并规范化内容

![](../images/model_io.jpeg)


In [1]:
# 去掉pip install 使用项目依赖版本
# ! pip install -U langchain

## 模型输入 Prompts

一个语言模型的提示是用户提供的一组指令或输入，用于引导模型的响应，帮助它理解上下文并生成相关和连贯的基于语言的输出，例如回答问题、完成句子或进行对话。


- 提示模板（Prompt Templates）：参数化的模型输入
- 示例选择器（Example Selectors）：动态选择要包含在提示中的示例


## 提示模板 Prompt Templates

**Prompt Templates 提供了一种预定义、动态注入、模型无关和参数化的提示词生成方式，以便在不同的语言模型之间重用模板。**

一个模板可能包括指令、少量示例以及适用于特定任务的具体背景和问题。

通常，提示要么是一个字符串（LLMs），要么是一组聊天消息（Chat Model）。


类继承关系:

```
BasePromptTemplate --> PipelinePromptTemplate
                       StringPromptTemplate --> PromptTemplate
                                                FewShotPromptTemplate
                                                FewShotPromptWithTemplates
                       BaseChatPromptTemplate --> AutoGPTPrompt
                                                  ChatPromptTemplate --> AgentScratchPadChatPromptTemplate



BaseMessagePromptTemplate --> MessagesPlaceholder
                              BaseStringMessagePromptTemplate --> ChatMessagePromptTemplate
                                                                  HumanMessagePromptTemplate
                                                                  AIMessagePromptTemplate
                                                                  SystemMessagePromptTemplate

PromptValue --> StringPromptValue
                ChatPromptValue
```


**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/prompts**



### 使用 PromptTemplate 类生成提升词

**通常，`PromptTemplate` 类的实例，使用Python的`str.format`语法生成模板化提示；也可以使用其他模板语法（例如jinja2）。**

#### 使用 from_template 方法实例化 PromptTemplate

In [2]:
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)

# 使用 format 生成提示
prompt = prompt_template.format(adjective="funny", content="chickens")
print(prompt)

Tell me a funny joke about chickens.


In [3]:
print(prompt_template)

input_variables=['adjective', 'content'] input_types={} partial_variables={} template='Tell me a {adjective} joke about {content}.'


In [4]:
prompt_template = PromptTemplate.from_template(
    "Tell me a joke"
)
# 生成提示
prompt = prompt_template.format()
print(prompt)

Tell me a joke


#### 使用构造函数（Initializer）实例化 PromptTemplate

使用构造函数实例化 `prompt_template` 时必须传入参数：`input_variables` 和 `template`。

在生成提示过程中，会检查输入变量与模板字符串中的变量是否匹配，如果不匹配，则会引发异常；

In [5]:
invalid_prompt = PromptTemplate(
    input_variables=["adjective"],
    template="Tell me a {adjective} joke about {content}."
)

传入 content 后才能生成可用的 prompt

In [6]:
valid_prompt = PromptTemplate(
    input_variables=["adjective", "content"],
    template="Tell me a {adjective} joke about {content}."
)

In [7]:
print(valid_prompt)

input_variables=['adjective', 'content'] input_types={} partial_variables={} template='Tell me a {adjective} joke about {content}.'


In [8]:
valid_prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens.'

In [9]:
prompt_template = PromptTemplate.from_template(
    "讲{num}个给程序员听得笑话"
)

In [10]:
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct", max_tokens=1000)

prompt = prompt_template.format(num=2)
print(f"prompt: {prompt}")

result = llm.invoke(prompt)
print(f"result: {result}")

In [11]:
# 【新增】ChatOpenAI调用方式，使用gpt-4o-mini模型
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-4o-mini", max_tokens=1000)

prompt = prompt_template.format(num=2)
print(f"prompt: {prompt}")

result = llm.invoke(prompt)
print(f"result: {result}")

prompt: 讲2个给程序员听得笑话
result: content='当然可以！给程序员的笑话如下：\n\n1. 为什么程序员总是喜欢在洗手间里工作？\n   因为那里有很多“控制”流（control flow）！\n\n2. 有一天，一个程序员走进了酒吧，点了一杯啤酒。酒保问他：“要不要加点柠檬？”程序员回答：“不，我只想要最简单的‘Hello, World!’”\n\n希望这些笑话能让你会心一笑！' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 17, 'total_tokens': 123, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': None, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': None}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_b705f0c291', 'finish_reason': 'stop', 'logprobs': None} id='run-3231f13c-95dc-4e93-ba30-3d5afdd03560-0' usage_metadata={'input_tokens': 17, 'output_tokens': 106, 'total_tokens': 123, 'input_token_details': {}, 'output_token_details': {}}


In [12]:
print(llm.invoke(prompt_template.format(num=3)))

content='当然可以！以下是三个适合程序员的笑话：\n\n1. **为什么程序员总是混淆万圣节和圣诞节？**\n   因为Oct 31 等于 Dec 25！\n\n2. **程序员的妻子对他生气了，问他为什么总是对代码那么上心。**\n   程序员回答：“因为它们比我们之间的沟通更容易调试！”\n\n3. **有什么是程序员最不喜欢的饮料？**\n   可乐，因为他们讨厌“糖”（bug）！\n\n希望你喜欢这些笑话！' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 131, 'prompt_tokens': 17, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': None, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': None}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_b705f0c291', 'finish_reason': 'stop', 'logprobs': None} id='run-2f34ac13-aadf-451a-ad5e-942a36737161-0' usage_metadata={'input_tokens': 17, 'output_tokens': 131, 'total_tokens': 148, 'input_token_details': {}, 'output_token_details': {}}


#### 使用 jinja2 生成模板化提示

In [13]:
jinja2_template = "Tell me a {{ adjective }} joke about {{ content }}"
prompt = PromptTemplate.from_template(jinja2_template, template_format="jinja2")

prompt.format(adjective="funny", content="chickens")

'Tell me a funny joke about chickens'

In [14]:
print(prompt)

input_variables=['adjective', 'content'] input_types={} partial_variables={} template='Tell me a {{ adjective }} joke about {{ content }}' template_format='jinja2'


#### 实测：生成多种编程语言版本的快速排序

In [15]:
sort_prompt_template = PromptTemplate.from_template(
    "生成可执行的快速排序 {programming_language} 代码"
)

In [16]:
print(llm.invoke(sort_prompt_template.format(programming_language="python")))

content='下面是一个简单的快速排序（Quicksort）算法的 Python 实现：\n\n```python\ndef quicksort(arr):\n    if len(arr) <= 1:\n        return arr\n    else:\n        pivot = arr[len(arr) // 2]  # 选择中间元素作为基准\n        left = [x for x in arr if x < pivot]  # 小于基准的元素\n        middle = [x for x in arr if x == pivot]  # 等于基准的元素\n        right = [x for x in arr if x > pivot]  # 大于基准的元素\n        return quicksort(left) + middle + quicksort(right)\n\n# 示例\nif __name__ == "__main__":\n    arr = [3, 6, 8, 10, 1, 2, 1]\n    sorted_arr = quicksort(arr)\n    print("排序后的数组:", sorted_arr)\n```\n\n在这个实现中，`quicksort` 函数接收一个数组 `arr`，并对其进行排序。选择数组中间的元素作为基准，然后分成三个部分：小于、等于和大于基准的元素，最后递归处理左边和右边的部分，并返回合并后的结果。\n\n你可以直接运行这段代码来查看排序效果。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 287, 'prompt_tokens': 16, 'total_tokens': 303, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'a

In [17]:
print(llm.invoke(sort_prompt_template.format(programming_language="java")))

content='下面是一个简单的快速排序（Quick Sort）算法的 Java 实现。快速排序是一种高效的排序算法，采用分治法策略，平均时间复杂度为 O(n log n)。\n\n```java\nimport java.util.Arrays;\n\npublic class QuickSort {\n\n    // 主函数\n    public static void main(String[] args) {\n        int[] arr = { 34, 7, 23, 32, 5, 62 };\n        System.out.println("原数组: " + Arrays.toString(arr));\n        quickSort(arr, 0, arr.length - 1);\n        System.out.println("排序后的数组: " + Arrays.toString(arr));\n    }\n\n    // 快速排序主方法\n    public static void quickSort(int[] arr, int low, int high) {\n        if (low < high) {\n            // 获取划分点\n            int partitionIndex = partition(arr, low, high);\n\n            // 递归排序左右两部分\n            quickSort(arr, low, partitionIndex - 1);\n            quickSort(arr, partitionIndex + 1, high);\n        }\n    }\n\n    // 划分方法\n    private static int partition(int[] arr, int low, int high) {\n        // 选择最后一个元素作为基准\n        int pivot = arr[high];\n        int i = (low - 1); // 小于基准的元素指针\n\n        for (int j = low; j < hi

In [18]:
print(llm.invoke(sort_prompt_template.format(programming_language="C++")))

content='下面是一个可执行的快速排序（Quick Sort）算法的 C++ 示例代码。此代码实现了快速排序，并包含一个简单的主函数可以进行测试。\n\n```cpp\n#include <iostream>\n#include <vector>\n\nusing namespace std;\n\n// 快速排序的划分函数\nint partition(vector<int>& arr, int low, int high) {\n    int pivot = arr[high];  // 选择最后一个元素作为基准\n    int i = (low - 1);      // 小于基准元素的索引\n\n    for (int j = low; j < high; j++) {\n        // 如果当前元素小于等于基准\n        if (arr[j] <= pivot) {\n            i++;  // 增加小于基准元素的索引\n            swap(arr[i], arr[j]);  // 交换元素\n        }\n    }\n    swap(arr[i + 1], arr[high]);  // 将基准元素放到正确位置\n    return i + 1;  // 返回基准元素的新索引\n}\n\n// 快速排序函数\nvoid quickSort(vector<int>& arr, int low, int high) {\n    if (low < high) {\n        int pi = partition(arr, low, high);  // 划分数组并获取基准元素的索引\n\n        quickSort(arr, low, pi - 1);  // 对基准左侧的子数组进行排序\n        quickSort(arr, pi + 1, high); // 对基准右侧的子数组进行排序\n    }\n}\n\n// 主函数\nint main() {\n    vector<int> arr = {10, 7, 8, 9, 1, 5};\n    int n = arr.size();\n\n    cout << "排序前的数组: ";\n    for (i

## 使用 ChatPromptTemplate 类生成适用于聊天模型的聊天记录

**`ChatPromptTemplate` 类的实例，使用`format_messages`方法生成适用于聊天模型的提示。**

### 使用 from_messages 方法实例化 ChatPromptTemplate

In [19]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

# 生成提示
messages = template.format_messages(
    name="Bob",
    user_input="What is your name?"
)

In [20]:
print(messages)

[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]


In [21]:
print(messages[0].content)
print(messages[-1].content)

You are a helpful AI bot. Your name is Bob.
What is your name?


In [22]:
from langchain_openai import ChatOpenAI
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=1000)

In [23]:
chat_model.invoke(messages)

AIMessage(content='My name is Bob.\n', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 51, 'total_tokens': 57, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': 0, 'reasoning_tokens': None, 'rejected_prediction_tokens': None, 'text_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0, 'text_tokens': 0, 'image_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run-56b4bc27-bb95-4687-87e1-001af8ff6f84-0', usage_metadata={'input_tokens': 51, 'output_tokens': 6, 'total_tokens': 57, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0}})

### 摘要总结

In [24]:
summary_template = ChatPromptTemplate.from_messages([
    ("system", "你将获得关于同一主题的{num}篇文章（用-----------标签分隔）。首先总结每篇文章的论点。然后指出哪篇文章提出了更好的论点，并解释原因。"),
    ("human", "{user_input}"),
])

In [25]:
messages = summary_template.format_messages(
    num=3,
    user_input='''1. [PHP是世界上最好的语言]
PHP是世界上最好的情感派编程语言，无需逻辑和算法，只要情绪。它能被蛰伏在冰箱里的PHP大神轻易驾驭，会话结束后的感叹号也能传达对代码的热情。写PHP就像是在做披萨，不需要想那么多，只需把配料全部扔进一个碗，然后放到服务器上，热乎乎出炉的网页就好了。
-----------
2. [Python是世界上最好的语言]
Python是世界上最好的拜金主义者语言。它坚信：美丽就是力量，简洁就是灵魂。Python就像是那个永远在你皱眉的那一刻扔给你言情小说的好友。只有Python，你才能够在两行代码之间感受到飘逸的花香和清新的微风。记住，这世上只有一种语言可以使用空格来领导全世界的进步，那就是Python。
-----------
3. [Java是世界上最好的语言]
Java是世界上最好的德育课编程语言，它始终坚守了严谨、安全的编程信条。Java就像一个严格的老师，他不会对你怀柔，不会让你偷懒，也不会让你走捷径，但他教会你规范和自律。Java就像是那个喝咖啡也算加班费的上司，拥有对邪恶的深度厌恶和对善良的深度拥护。
'''
)

In [26]:
print(messages[-1].content)

1. [PHP是世界上最好的语言]
PHP是世界上最好的情感派编程语言，无需逻辑和算法，只要情绪。它能被蛰伏在冰箱里的PHP大神轻易驾驭，会话结束后的感叹号也能传达对代码的热情。写PHP就像是在做披萨，不需要想那么多，只需把配料全部扔进一个碗，然后放到服务器上，热乎乎出炉的网页就好了。
-----------
2. [Python是世界上最好的语言]
Python是世界上最好的拜金主义者语言。它坚信：美丽就是力量，简洁就是灵魂。Python就像是那个永远在你皱眉的那一刻扔给你言情小说的好友。只有Python，你才能够在两行代码之间感受到飘逸的花香和清新的微风。记住，这世上只有一种语言可以使用空格来领导全世界的进步，那就是Python。
-----------
3. [Java是世界上最好的语言]
Java是世界上最好的德育课编程语言，它始终坚守了严谨、安全的编程信条。Java就像一个严格的老师，他不会对你怀柔，不会让你偷懒，也不会让你走捷径，但他教会你规范和自律。Java就像是那个喝咖啡也算加班费的上司，拥有对邪恶的深度厌恶和对善良的深度拥护。



In [27]:
chat_result = chat_model.invoke(messages)

In [28]:
print(chat_result.content)

好的，我将按照你的要求，对这三篇文章进行分析和比较。

**文章1：PHP是世界上最好的语言**

*   **论点总结：** 这篇文章以一种戏谑和夸张的方式，表达了对PHP的喜爱。它强调PHP的易用性，认为PHP编程不需要过多的逻辑和算法，更像是一种情感的表达。文章将PHP编程比作制作披萨，突出其简单和快速的特点。

-----------

**文章2：Python是世界上最好的语言**

*   **论点总结：** 这篇文章同样以一种幽默的方式赞扬了Python。它强调Python的简洁和优雅，认为Python是一种充满美感的编程语言。文章将Python比作一个善解人意的朋友，突出其易学易用的特点，并暗示Python在推动世界进步中的作用。

-----------

**文章3：Java是世界上最好的语言**

*   **论点总结：** 这篇文章以一种严肃的态度赞扬了Java。它强调Java的严谨和安全，认为Java是一种规范和自律的编程语言。文章将Java比作一个严格的老师和一个负责的上司，突出其在软件开发中的稳定性和可靠性。

-----------

**哪篇文章提出了更好的论点？**

这三篇文章都使用了幽默和拟人的手法来表达对特定编程语言的喜爱，但都没有提供具体的、有说服力的论据来支持自己的观点。因此，很难说哪篇文章提出了更好的论点。

如果要选择一个，我认为**第三篇关于Java的文章**略胜一筹，因为它在幽默之外，还提到了Java的一些实际优点，例如严谨、安全、规范和自律。虽然这些优点并没有展开论述，但至少为读者提供了一些思考的方向。

总的来说，这三篇文章更像是对不同编程语言的“爱发电”，而不是严谨的论证。它们旨在博君一笑，而非进行深入的讨论。



In [29]:
messages = summary_template.format_messages(
    num=2,
    user_input='''1.认为“道可道”中的第一个“道”，指的是道理，如仁义礼智之类；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，所谓“道可道，非常道”，指的是可以言说的道理，不是恒久存在的“道”，恒久存在的“道”不可言说。如苏辙说：“莫非道也。而可道者不可常，惟不可道，而后可常耳。今夫仁义礼智，此道之可道者也。然而仁不可以为义，而礼不可以为智，可道之不可常如此。……而道常不变，不可道之能常如此。”蒋锡昌说：“此道为世人所习称之道，即今人所谓‘道理’也，第一‘道’字应从是解。《广雅·释诂》二：‘道，说也’，第二‘道’字应从是解。‘常’乃真常不易之义，在文法上为区别词。……第三‘道’字即二十五章‘道法自然’之‘道’，……乃老子学说之总名也”。陈鼓应说：“第一个‘道’字是人们习称之道，即今人所谓‘道理’。第二个‘道’字，是指言说的意思。第三个‘道’字，是老子哲学上的专有名词，在本章它意指构成宇宙的实体与动力。……‘常道’之‘常’，为真常、永恒之意。……可以用言词表达的道，就不是常道”。
-----------
2.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，“道可道，非常道”，指可以言说的“道”，就不是恒久存在的“道”。如张默生说：“‘道’，指宇宙的本体而言。……‘常’，是经常不变的意思。……可以说出来的道，便不是经常不变的道”。董平说：“第一个‘道’字与‘可道’之‘道’，内涵并不相同。第一个‘道’字，是老子所揭示的作为宇宙本根之‘道’；‘可道’之‘道’，则是‘言说’的意思。……这里的大意就是说：凡一切可以言说之‘道’，都不是‘常道’或永恒之‘道’”。汤漳平等说：“第一句中的三个‘道’，第一、三均指形上之‘道’，中间的‘道’作动词，为可言之义。……道可知而可行，但非恒久不变之道”。
--------
3.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“可道”中的“道”，指言说的意思；“常道”，则指的是平常人所讲之道、常俗之道。因此，“道可道，非常道”，指“道”是可以言说的，但它不是平常人所谓的道或常俗之道。如李荣说：“道者，虚极之理也。夫论虚极之理，不可以有无分其象，不可以上下格其真。……圣人欲坦兹玄路，开以教门，借圆通之名，目虚极之理，以理可名，称之可道。故曰‘吾不知其名，字之曰道’。非常道者，非是人间常俗之道也。人间常俗之道，贵之以礼义，尚之以浮华，丧身以成名，忘己而徇利。”司马光说：“世俗之谈道者，皆曰道体微妙，不可名言。老子以为不然，曰道亦可言道耳，然非常人之所谓道也。……常人之所谓道者，凝滞于物。”裘锡圭说：“到目前为止，可以说，几乎从战国开始，大家都把‘可道’之‘道’……看成老子所否定的，把‘常道’‘常名’看成老子所肯定的。这种看法其实有它不合理的地方，……‘道’是可以说的。《老子》这个《道经》第一章，开宗明义是要讲他的‘道’。第一个‘道’字，理所应当，也是讲他要讲的‘道’：道是可以言说的。……那么这个‘恒’字应该怎么讲？我认为很简单，‘恒’字在古代作定语用，经常是‘平常’‘恒常’的意思。……‘道’是可以言说的，但是我要讲的这个‘道’，不是‘恒道’，它不是一般人所讲的‘道’。
'''
)
print(messages)

[SystemMessage(content='你将获得关于同一主题的2篇文章（用-----------标签分隔）。首先总结每篇文章的论点。然后指出哪篇文章提出了更好的论点，并解释原因。', additional_kwargs={}, response_metadata={}), HumanMessage(content='1.认为“道可道”中的第一个“道”，指的是道理，如仁义礼智之类；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，所谓“道可道，非常道”，指的是可以言说的道理，不是恒久存在的“道”，恒久存在的“道”不可言说。如苏辙说：“莫非道也。而可道者不可常，惟不可道，而后可常耳。今夫仁义礼智，此道之可道者也。然而仁不可以为义，而礼不可以为智，可道之不可常如此。……而道常不变，不可道之能常如此。”蒋锡昌说：“此道为世人所习称之道，即今人所谓‘道理’也，第一‘道’字应从是解。《广雅·释诂》二：‘道，说也’，第二‘道’字应从是解。‘常’乃真常不易之义，在文法上为区别词。……第三‘道’字即二十五章‘道法自然’之‘道’，……乃老子学说之总名也”。陈鼓应说：“第一个‘道’字是人们习称之道，即今人所谓‘道理’。第二个‘道’字，是指言说的意思。第三个‘道’字，是老子哲学上的专有名词，在本章它意指构成宇宙的实体与动力。……‘常道’之‘常’，为真常、永恒之意。……可以用言词表达的道，就不是常道”。\n-----------\n2.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，“道可道，非常道”，指可以言说的“道”，就不是恒久存在的“道”。如张默生说：“‘道’，指宇宙的本体而言。……‘常’，是经常不变的意思。……可以说出来的道，便不是经常不变的道”。董平说：“第一个‘道’字与‘可道’之‘道’，内涵并不相同。第一个‘道’字，是老子所揭示的作为宇宙本根之‘道’；‘可道’之‘道’，则是‘言说’的意思。……这里的大意就是说：凡一切可以言说之‘道’，都不是‘常道’或永恒之‘道’”。汤漳平等说：“第一句中的三个‘道’，第一、三均指形上之‘道’，中间的‘道’作动词，为可言之义。……道可知而可行，但非恒久不变之道”。\n--------\n3.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“

In [30]:
chat_result = chat_model.invoke(messages)
print(chat_result.content)

好的，以下是针对你提供的三篇文章的分析：

**文章一：**

*   **论点总结：**
    *   第一个“道”指的是道理，如仁义礼智等。
    *   “可道”中的“道”指的是言说。
    *   “常道”指的是恒久存在的道，是不可言说的。
    *   因此，“道可道，非常道”指的是可以言说的道理不是恒久存在的道，恒久存在的道不可言说。
    *   通过引用苏辙、蒋锡昌、陈鼓应等人的观点来支持这一论点。
*   **主要论据：**
    *   引用《广雅·释诂》来解释“道”有“说”的意思。
    *   通过分析“常”字的含义，强调“常道”的真常、永恒之意。

**文章二：**

*   **论点总结：**
    *   第一个“道”指的是宇宙万物的本原。
    *   “可道”中的“道”指的是言说。
    *   “常道”指的是恒久存在的道。
    *   因此，“道可道，非常道”指的是可以言说的道就不是恒久存在的道。
    *   通过引用张默生、董平、汤漳平等人的观点来支持这一论点。
*   **主要论据：**
    *   强调第一个“道”与“可道”之“道”的内涵不同，前者是宇宙本根，后者是言说。
    *   通过解释“常”的含义，强调“常道”的经常不变。

**文章三：**

*   **论点总结：**
    *   第一个“道”指的是宇宙万物的本原。
    *   “可道”中的“道”指的是言说。
    *   “常道”指的是平常人所讲的道、常俗之道。
    *   因此，“道可道，非常道”指的是“道”是可以言说的，但它不是平常人所谓的道或常俗之道。
    *   通过引用李荣、司马光、裘锡圭等人的观点来支持这一论点。
*   **主要论据：**
    *   引用李荣的观点，将“道”解释为“虚极之理”，并认为圣人借“道”来开示教门。
    *   引用司马光的观点，认为老子认为“道”是可以言说的，但不是常人所理解的“道”。
    *   引用裘锡圭的观点，认为“恒”字有“平常”、“恒常”的意思，因此“常道”指的是平常人所讲的道。

**哪个论点更好？**

我认为第三篇文章提出了更好的论点。原因如下：

1.  **更贴合《老子》的整体思想：**第三篇文章认为“常道”指的是平常人所讲的道，这与老

In [31]:
messages = summary_template.format_messages(
    num=2,
    user_input='''1.认为“道可道”中的第一个“道”，指的是道理，如仁义礼智之类；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，所谓“道可道，非常道”，指的是可以言说的道理，不是恒久存在的“道”，恒久存在的“道”不可言说。如苏辙说：“莫非道也。而可道者不可常，惟不可道，而后可常耳。今夫仁义礼智，此道之可道者也。然而仁不可以为义，而礼不可以为智，可道之不可常如此。……而道常不变，不可道之能常如此。”蒋锡昌说：“此道为世人所习称之道，即今人所谓‘道理’也，第一‘道’字应从是解。《广雅·释诂》二：‘道，说也’，第二‘道’字应从是解。‘常’乃真常不易之义，在文法上为区别词。……第三‘道’字即二十五章‘道法自然’之‘道’，……乃老子学说之总名也”。陈鼓应说：“第一个‘道’字是人们习称之道，即今人所谓‘道理’。第二个‘道’字，是指言说的意思。第三个‘道’字，是老子哲学上的专有名词，在本章它意指构成宇宙的实体与动力。……‘常道’之‘常’，为真常、永恒之意。……可以用言词表达的道，就不是常道”。
-----------
2.认为“道可道”中的第一个“道”，指的是宇宙万物的本原；“可道”中的“道”，指言说的意思；“常道”，指恒久存在的“道”。因此，“道可道，非常道”，指可以言说的“道”，就不是恒久存在的“道”。如张默生说：“‘道’，指宇宙的本体而言。……‘常’，是经常不变的意思。……可以说出来的道，便不是经常不变的道”。董平说：“第一个‘道’字与‘可道’之‘道’，内涵并不相同。第一个‘道’字，是老子所揭示的作为宇宙本根之‘道’；‘可道’之‘道’，则是‘言说’的意思。……这里的大意就是说：凡一切可以言说之‘道’，都不是‘常道’或永恒之‘道’”。汤漳平等说：“第一句中的三个‘道’，第一、三均指形上之‘道’，中间的‘道’作动词，为可言之义。……道可知而可行，但非恒久不变之道”。
'''
)

In [32]:
chat_result = chat_model.invoke(messages)
print(chat_result.content)

好的，我将根据您提供的两段内容进行分析。

**文章一总结：**

这篇文章认为“道可道”中的第一个“道”指的是道理，类似于仁义礼智等规范和原则；“可道”中的“道”指的是言说的行为；“常道”指的是恒久存在的“道”。 因此， “道可道，非常道”意味着可以用语言表达的道理，并非永恒不变的道。 引用了苏辙、蒋锡昌和陈鼓应等人的观点来支持这一解释，他们都倾向于将第一个“道”理解为人们常说的道理或规范。

**文章二总结：**

这篇文章认为“道可道”中的第一个“道”指的是宇宙万物的本原；“可道”中的“道”指言说的意思；“常道”指恒久存在的“道”。 因此， “道可道，非常道”指可以用言语表达的“道”，就不是永恒的“道”。 引用了张默生、董平和汤漳平等人的观点来支持这一解释，他们都倾向于将第一个“道”理解为宇宙本根或本体。

**哪篇文章提出了更好的论点？**

我认为**文章二**提出了更好的论点。理由如下：

*   **更符合《道德经》的哲学体系：** 老子《道德经》的核心在于探讨宇宙本源和运行规律，而不仅仅是伦理道德规范。将第一个“道”理解为宇宙万物的本原，更能体现《道德经》的深刻性和哲学高度。
*   **更贴合“常道”的含义：** “常道”指的是永恒不变的道。 如果将第一个“道”理解为仁义礼智等，这些概念在不同时代和社会背景下会有不同的解释和表现形式，不符合“常道”的“常”的含义。 而宇宙本源的道，更具有永恒性和普遍性。
*   **逻辑更为一致：** 将第一个“道”理解为宇宙本源，与后面“可道”的“道”形成对比，强调了语言的局限性，即语言无法完全表达宇宙本源的道的深刻含义。 这种对比更能突出《道德经》对于超越语言、体悟大道的追求。
*   **虽然两篇文章都引用了不同的学者观点，但文章二的解释更符合对《道德经》主流的理解和解读。**

综上所述，虽然将第一个“道”理解为“道理”也有其一定的道理，但从哲学体系、概念贴合度以及逻辑一致性等方面考虑，将第一个“道”理解为宇宙万物的本原，更能体现《道德经》的思想精髓。




### 使用 FewShotPromptTemplate 类生成 Few-shot Prompt 

构造 few-shot prompt 的方法通常有两种：
- 从示例集（set of examples）中手动选择；
- 通过示例选择器（Example Selector）自动选择.

In [33]:
from langchain.prompts.prompt import PromptTemplate


examples = [
  {
    "question": "谁活得更久，穆罕默德·阿里还是艾伦·图灵？",
    "answer": 
"""
这里需要进一步的问题吗：是的。
追问：穆罕默德·阿里去世时多大了？
中间答案：穆罕默德·阿里去世时74岁。
追问：艾伦·图灵去世时多大了？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里
"""
  },
  {
    "question": "craigslist的创始人是什么时候出生的？",
    "answer": 
"""
这里需要进一步的问题吗：是的。
追问：谁是craigslist的创始人？
中间答案：Craigslist是由Craig Newmark创办的。
追问：Craig Newmark是什么时候出生的？
中间答案：Craig Newmark出生于1952年12月6日。
所以最终答案是：1952年12月6日
"""
  },
  {
    "question": "乔治·华盛顿的外祖父是谁？",
    "answer":
"""
这里需要进一步的问题吗：是的。
追问：谁是乔治·华盛顿的母亲？
中间答案：乔治·华盛顿的母亲是Mary Ball Washington。
追问：Mary Ball Washington的父亲是谁？
中间答案：Mary Ball Washington的父亲是Joseph Ball。
所以最终答案是：Joseph Ball
"""
  },
  {
    "question": "《大白鲨》和《皇家赌场》的导演是同一个国家的吗？",
    "answer":
"""
这里需要进一步的问题吗：是的。
追问：谁是《大白鲨》的导演？
中间答案：《大白鲨》的导演是Steven Spielberg。
追问：Steven Spielberg来自哪里？
中间答案：美国。
追问：谁是《皇家赌场》的导演？
中间答案：《皇家赌场》的导演是Martin Campbell。
追问：Martin Campbell来自哪里？
中间答案：新西兰。
所以最终答案是：不是
"""
  }
]

In [34]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\n{answer}"
)

# **examples[0] 是将examples[0] 字典的键值对（question-answer）解包并传递给format，作为函数参数
print(example_prompt.format(**examples[0]))

Question: 谁活得更久，穆罕默德·阿里还是艾伦·图灵？

这里需要进一步的问题吗：是的。
追问：穆罕默德·阿里去世时多大了？
中间答案：穆罕默德·阿里去世时74岁。
追问：艾伦·图灵去世时多大了？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里



In [35]:
print(example_prompt)

input_variables=['answer', 'question'] input_types={} partial_variables={} template='Question: {question}\n{answer}'


In [36]:
print(example_prompt.format(**examples[-1]))

Question: 《大白鲨》和《皇家赌场》的导演是同一个国家的吗？

这里需要进一步的问题吗：是的。
追问：谁是《大白鲨》的导演？
中间答案：《大白鲨》的导演是Steven Spielberg。
追问：Steven Spielberg来自哪里？
中间答案：美国。
追问：谁是《皇家赌场》的导演？
中间答案：《皇家赌场》的导演是Martin Campbell。
追问：Martin Campbell来自哪里？
中间答案：新西兰。
所以最终答案是：不是



#### 关于解包的示例

In [37]:
def print_info(question, answer):
    print(f"Question: {question}")
    print(f"Answer: {answer}")

print_info(**examples[0]) 

Question: 谁活得更久，穆罕默德·阿里还是艾伦·图灵？
Answer: 
这里需要进一步的问题吗：是的。
追问：穆罕默德·阿里去世时多大了？
中间答案：穆罕默德·阿里去世时74岁。
追问：艾伦·图灵去世时多大了？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里



### 生成 Few-shot Prompt

In [38]:
# 导入 FewShotPromptTemplate 类
from langchain.prompts.few_shot import FewShotPromptTemplate

# 创建一个 FewShotPromptTemplate 对象
few_shot_prompt = FewShotPromptTemplate(
    examples=examples,           # 使用前面定义的 examples 作为范例
    example_prompt=example_prompt, # 使用前面定义的 example_prompt 作为提示模板
    suffix="Question: {input}",    # 后缀模板，其中 {input} 会被替换为实际输入
    input_variables=["input"]     # 定义输入变量的列表
)

# 使用给定的输入格式化 prompt，并打印结果
# 这里的 {input} 将被 "玛丽·波尔·华盛顿的父亲是谁?" 替换
print(few_shot_prompt.format(input="玛丽·波尔·华盛顿的父亲是谁?"))

Question: 谁活得更久，穆罕默德·阿里还是艾伦·图灵？

这里需要进一步的问题吗：是的。
追问：穆罕默德·阿里去世时多大了？
中间答案：穆罕默德·阿里去世时74岁。
追问：艾伦·图灵去世时多大了？
中间答案：艾伦·图灵去世时41岁。
所以最终答案是：穆罕默德·阿里


Question: craigslist的创始人是什么时候出生的？

这里需要进一步的问题吗：是的。
追问：谁是craigslist的创始人？
中间答案：Craigslist是由Craig Newmark创办的。
追问：Craig Newmark是什么时候出生的？
中间答案：Craig Newmark出生于1952年12月6日。
所以最终答案是：1952年12月6日


Question: 乔治·华盛顿的外祖父是谁？

这里需要进一步的问题吗：是的。
追问：谁是乔治·华盛顿的母亲？
中间答案：乔治·华盛顿的母亲是Mary Ball Washington。
追问：Mary Ball Washington的父亲是谁？
中间答案：Mary Ball Washington的父亲是Joseph Ball。
所以最终答案是：Joseph Ball


Question: 《大白鲨》和《皇家赌场》的导演是同一个国家的吗？

这里需要进一步的问题吗：是的。
追问：谁是《大白鲨》的导演？
中间答案：《大白鲨》的导演是Steven Spielberg。
追问：Steven Spielberg来自哪里？
中间答案：美国。
追问：谁是《皇家赌场》的导演？
中间答案：《皇家赌场》的导演是Martin Campbell。
追问：Martin Campbell来自哪里？
中间答案：新西兰。
所以最终答案是：不是


Question: 玛丽·波尔·华盛顿的父亲是谁?


## 示例选择器 Example Selectors

**如果你有大量的参考示例，就得选择哪些要包含在提示中。最好还是根据某种条件或者规则来自动选择，Example Selector 是负责这个任务的类。**

BaseExampleSelector 定义如下：

```python
class BaseExampleSelector(ABC):
    """用于选择包含在提示中的示例的接口。"""

    @abstractmethod
    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """根据输入选择要使用的示例。"""

```

`ABC` 是 Python 中的 `abc` 模块中的一个缩写，它表示 "Abstract Base Class"（抽象基类）。在 Python 中，抽象基类用于定义其他类必须遵循的基本接口或蓝图，但不能直接实例化。其主要目的是为了提供一种形式化的方式来定义和检查子类的接口。

使用抽象基类的几点关键信息：

1. **抽象方法**：在抽象基类中，你可以定义抽象方法，它没有实现（也就是说，它没有方法体）。任何继承该抽象基类的子类都必须提供这些抽象方法的实现。

2. **不能直接实例化**：你不能直接创建抽象基类的实例。试图这样做会引发错误。它们的主要目的是为了被继承，并在子类中实现其方法。

3. **强制子类实现**：如果子类没有实现所有的抽象方法，那么试图实例化该子类也会引发错误。这确保了继承抽象基类的所有子类都遵循了预定的接口。


In [39]:
# 导入需要的模块和类
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# 定义一个提示模板
example_prompt = PromptTemplate(
    input_variables=["input", "output"],     # 输入变量的名字
    template="Input: {input}\nOutput: {output}",  # 实际的模板字符串
)

# 这是一个假设的任务示例列表，用于创建反义词
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

### Pandas 相关包首次导入错误后，再次执行即可正确导入

In [40]:

# 从给定的示例中创建一个语义相似性选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,                          # 可供选择的示例列表
    OpenAIEmbeddings(),                # 用于生成嵌入向量的嵌入类，用于衡量语义相似性
    Chroma,                            # 用于存储嵌入向量并进行相似性搜索的 VectorStore 类
    k=1                                # 要生成的示例数量
)

# 创建一个 FewShotPromptTemplate 对象
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,  # 提供一个 ExampleSelector 替代示例
    example_prompt=example_prompt,      # 前面定义的提示模板
    prefix="Give the antonym of every input", # 前缀模板
    suffix="Input: {adjective}\nOutput:",     # 后缀模板
    input_variables=["adjective"],           # 输入变量的名字
)


In [41]:
# 输入是一种感受，所以应该选择 happy/sad 的示例。
print(similar_prompt.format(adjective="worried"))

Give the antonym of every input

Input: happy
Output: sad

Input: worried
Output:


In [42]:
# 输入是一种度量，所以应该选择 tall/short的示例。
print(similar_prompt.format(adjective="long"))

Give the antonym of every input

Input: tall
Output: short

Input: long
Output:


In [43]:
print(similar_prompt.format(adjective="rain"))

Give the antonym of every input

Input: windy
Output: calm

Input: rain
Output:
